# 1. matplotlib vs LaTeX

In [ ]:
# Some useful libs for data wrangling
import numpy as np  # Scientific computing
import scipy  # Scientific and technical computing
import pandas as pd  # (Tabular) data manipulation and analysis
import statsmodels as sm  # -> Statistical modeling

# And some for visualization
import matplotlib as mpl  # -> Visualization with Python (inspired by MATLAB)
from matplotlib import pyplot as plt  # -> An interactive way to work with matplotlib
from matplotlib import pylab  # -> Same as PyPlot but some additional shortcuts (use is discouraged now)
import seaborn as sns  # -> Statistical data visualization
import colorcet as cc  # -> Collection of perceptually accurate colormaps

%matplotlib inline

### 1.1 Matplotlib madness

In [ ]:
fig = mpl.figure.Figure()  # -> Figure object: the canvas
fig = plt.figure()
ax = fig.gca()  # -> Axes object(s): Entry point to access (modify/add/remove) content
ax = fig.subplots()
fig, ax = plt.subplots()

### 1.2 Basic figure setup

In [ ]:
fig, ax = plt.subplots()

# fig.set_size_inches((12, 9))
# fig.set_dpi(300)  # -> Dots per inch
# fig.set_tight_layout(True)

# ax.spines.top.set_visible(False)
# ax.spines.right.set_visible(False)
# ax.tick_params(labelsize=14)

plt.show()  # -> Display all open figures
# fig.show()  # -> Display this figure

#### 1.2.1 Set size in dots (pixels) instead of inches

In [ ]:
width, height = 640, 480
dpi = 300
inches_per_dot = 1 / dpi
width_inch, height_inch = width * inches_per_dot, height * inches_per_dot

fig, ax = plt.subplots(figsize=(width_inch, height_inch), dpi=dpi, tight_layout=True)

### 1.3 Exporting as image
* Familiar format
* Easy to view/interact with
* Can be directly used with `\includegraphics{figure.png}` in LaTeX

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9), dpi=300, tight_layout=True)

# Put your figure code here
ax.set_ylabel(r'$\log N$', fontsize=16)
ax.set_xlabel(r'$\log\tau$', fontsize=16)

plt.savefig("figure.png", format="png", bbox_inches="tight", pad_inches=0)

### 1.4 Export as PDF
* Lossless
* Can also be directly used with `\includegraphics{figure.pdf}` in LaTeX

In [ ]:
plt.savefig("figure.pdf", format="pdf", dpi=300)

### 1.5 Export as SVG
* Vector format (infinite resolution)
* Needs additional packages for LaTeX import
* Alternative: Convert to PDF
  * `inkscape -D image.svg -o image.pdf --export-latex`
  * This puts the figure text into a .tex file for proper typesetting in LaTeX

In [ ]:
plt.savefig("figure.svg", format="svg", dpi=300)

### 1.6 Export as PGF
* Lossless
* Figure text font matches document
* Text is searchable (as it is actually text)
* Smaller file size
* Increased compile time/memory
* Can be directly used with `\input{figure.pgf}` in LaTeX
* Use `\showthe\textwidth` to get the width of your LaTeX text in pixel as reference for your figure

In [ ]:
mpl.use("pgf")
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False,
})
plt.savefig("figure.svg", format="pgf", backend="pgf", dpi=300)

# 2. Interactive 2D & 3D plots with Plotly

In [ ]:
df = pd.read_csv("../data/bugs.txt", sep='\t')
python = df.groupby('group').get_group('Python')
scheme = df.groupby('group').get_group('Scheme')

dpi = 100
fig, ax = plt.subplots(figsize=(900 / dpi, 600 / dpi), dpi=dpi, tight_layout=True)

ax.spines.top.set_visible(False)
ax.spines.right.set_visible(False)

data = python.sort_values('lines')
xlimits = [0, 55]
ylimits = [155, 185]
xstep = 5
ystep = 5

x = np.arange(1.5, 76.5, 1.5)
m, b = np.polyfit(x, data['bugs'].values, 1)
ax.scatter(x, data['bugs'], facecolor=[(31 / 255, 119 / 255, 180 / 255, 1)], s=12)
ax.plot(x, m * x + b, color=(31 / 255, 119 / 255, 180 / 255, 1), linestyle="--", label="Python")

x = np.arange(1, 51, 1)
m, b = np.polyfit(x, data['bugs'].values, 1)
ax.scatter(x, data['bugs'], facecolor=[(255 / 255, 187 / 255, 120 / 255, 1)], s=12)
ax.plot(x, m * x + b, color=(255 / 255, 187 / 255, 120 / 255, 1), linestyle="--", label="Scheme")

ax.set_xlim(xlimits)
ax.set_ylim(ylimits)
plt.xticks(np.arange(xlimits[0], xlimits[1] + 1, xstep), fontsize=10)
ax.set_xlabel('Lines of code', fontsize=12, labelpad=20)
ax.set_ylabel('Number of bugs', fontsize=12, labelpad=20)
ax.spines.left.set_position(('axes', -0.05))
ax.spines.bottom.set_position(('axes', -0.05))
plt.yticks(np.arange(ylimits[0], ylimits[1] + 1, ystep), fontsize=10)
ax.legend(frameon=False, loc="lower right")
plt.show()

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
from plotly.colors import qualitative

colors = qualitative.Plotly
fig = go.Figure()

x = np.arange(1.5, 76.5, 1.5)
m, b = np.polyfit(x, data["bugs"].values, 1)
fig.add_trace(go.Scatter(name="Python",
                         x=x,
                         y=m * x + b,
                         hovertemplate="%{y:.2f}",
                         mode="lines",
                         marker_color=colors[0],
                         line=dict(dash="dash", width=2)))
fig.add_trace(go.Scatter(x=x,
                         y=data["bugs"],
                         mode="markers",
                         marker=dict(color=colors[0], size=7),
                         showlegend=False,
                         hoverinfo="none"))

x = np.arange(1, 51, 1)
m, b = np.polyfit(x, data["bugs"].values, 1)
fig.add_trace(go.Scatter(name="Scheme",
                         x=x,
                         y=m * x + b,
                         hovertemplate="%{y:.2f}",
                         mode="lines",
                         marker_color=colors[4],
                         line=dict(dash="dash", width=2)))
fig.add_trace(go.Scatter(x=x,
                         y=data["bugs"],
                         mode="markers",
                         marker=dict(color=colors[4], size=7),
                         showlegend=False,
                         hoverinfo="none"))

fig.update_layout(xaxis=dict(
                    zeroline=False,
                    tick0=0,
                    dtick=5,
                    range=[0, 55],
                    constrain="domain",
                    showgrid=False,
                    title="Lines of code",
                    titlefont_size=16,
                    tickfont_size=14),
                yaxis=dict(
                    zeroline=False,
                    tick0=155,
                    dtick=5,
                    range=[155, 185],
                    constrain="domain",
                    showgrid=False,
                    title="Number of bugs",
                    titlefont_size=16,
                    tickfont_size=14),
                legend=dict(
                    font=dict(size=16),
                    xanchor="right",
                    yanchor="bottom",
                    y=0.1,
                    bgcolor='rgba(0, 0, 0, 0)',
                    bordercolor='rgba(0, 0, 0, 0)'),
                  template="plotly_white",
                  hoverlabel=dict(font_size=14),
                  hovermode="x",
                  width=900,
                  height=600,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0))
fig.show()

In [ ]:
pio.write_html(fig,
               file="figure.html",
               config=dict(displayModeBar=False),
               full_html=False,
               include_plotlyjs=True)